** The class examples in this notebook are my own. 

# Classes

#### The instantiation operation (“calling” a class object) creates an empty object. Many classes like to create objects with instances customized to a specific initial state. Therefore a class may define a special method named __init__(), like this:

In [1]:
# a simple class example 

class parameters:
    def __init__(self, alpha, beta, epeak, norm):
        self.alpha = alpha
        self.beta = beta
        self.epeak = epeak
        self.norm = norm

In [2]:
pars = parameters(-1.25, -2.74, 531.68, 0.07529)

In [3]:
pars.alpha

-1.25

In [4]:
pars.beta

-2.74

In [6]:
class Models(object):
    
    def __init__(self, modelname):
        self.modelname = modelname
    
    def band(self, alpha, beta, epeak, norm):
        self.alpha = -1.0
        self.beta = -2.0
        self.epeak = 300.0
        self.norm = 0.01
        
        def pars(self, x):
            vals = x*4
            return vals

In [8]:
k = Models('band')

TypeError: band() takes exactly 5 arguments (1 given)

In [34]:
from __future__ import division, print_function
import sys, os, glob
import numpy
import json
import pandas
from astropy.io import fits as pyfits
from collections import OrderedDict


old_stdout = sys.stdout

# Disable Print
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore Print
def enablePrint():
    sys.stdout = old_stdout


class Data(object):
    '''
    Data(**kwargs)
        where the kwargs represent the following dictionary:
    
    
    params_dict = {
            'burst':        'bn080916009',
            'det':          'L',
            'model':        'grbm',
            'pyx_version':  '-01-',
            'bxa_version':  '-01-',
            'bxa_Aversion': '-01-'
            }
                    
    test        = Data(**params_dict)
    


    burst:   str, burst trigger name.  ex: 'bn080916009'
    det:     str, 'L' or 'G'
    model:   str, full model name.  ex: 'grbm+blackb+lpow'
    pyx_version:  str, PYXSPEC version with '-' around it.  ex: '-01-'
    bxa_version:  str, BXA Orig version with '-' around it.  ex: '-01-'
    bxa_Aversion:  str, BXA Auto version with '-' around it.  ex: '-01-'


    EXAMPLES:

            example 1:
    use *args if using this class as a standalone.
      pars = ['bn080916009', 'G', 'grbm', '-01-', '-01-', '-01-']
      Data(*pars)
    The downfall of the *args method is that the variables in the
    list must always be in the same order and contain all the same
    variables.

            example 2:
    use **kwargs if inheriting the attributes of this class.
    This will be done WITHIN another class.
      pars = {
            'burst':        'bn080916009',
            'det':          'G',
            'model':        'grbm',
            'pyx_version':  '-01-',
            'bxa_version':  '-01-',
            'bxa_Aversion': '-01-'
            }
            
      Data(**pars)

    Within this class, there will be:
    self._burst = kwargs['burst']
    self._det    = kwargs['det']
    etc...

    The pars dictionary can also be written in the following way if
    it already inherited the variables into the self variable:
      pars = {'burst':self._burst, 'det':self._det,
      'model':self._model, 'pyx_version': self._pyx_version,
      'bxa_version': self._bxa_version,
      'bxa_Aversion':self._bxa_Aversion}

    The nice thing about the dictionary is that the entries do NOT need to
    be in any specific order and you can pass more than needed by the
    class or function that is inheriting them.

    kwargs['burst'] will find the burst dictionary entry.


    '''

    
    def __init__(self, *args, **kwargs):
        if args:
            raise Exception("Use Dictionary for now.")
        elif kwargs:
            self._burst           = kwargs['burst']
            self._det             = kwargs['det']
            if 'model' in kwargs.keys():
                self._model           = kwargs['model']
            if 'pyx_version' in kwargs.keys():
                self._pyx_version     = kwargs['pyx_version']
            if 'bxa_version' in kwargs.keys():
                self._bxa_version     = kwargs['bxa_version']
            self._bxa_Aversion    = '-01-'
            
        # DEFINE A FEW DIRECTORIES 
        self._det_dir             = ('GBMwLAT' if 'L' in self._det else 'GBM')
        self._base_dir            = "/Users/KimiZ/Python/My_Modules/GRBAnalysis/"
        self._basedata_dir        = os.path.join(self._base_dir, 'data/')
        self._burst_dir           = "/Users/KimiZ/GRBs2/analysis/LAT/%s/"%(self._burst)
        

    # FILE PREFIX IF THE BXA PROGRAMMING RESULTS ARE DESIRED.
    # FOR AUTO RUNS, AN EXAMPLE WOULD BE:        grbm_-01-_auto_G_
    # FOR ORIGINAL RUNS, AND EXAMPLE WOULD BE:   grbm_-01-_G_
    def _Bxa_Prefix(self, method):
        if 'a' in method:
            return "%s_-01-_auto_%s_"%(self._model, self._det) # grbm_-01-_auto_G_
        elif 'o' in method:
            return "%s_%s_%s_"%(self._model, self._bxa_version, self._det) # grbm_-01-_G_

    # FILE PREFIX FOR THE PYXSPEC PROGRAMMING FILES (PYTHON WRAPPER OF XSPEC)
    def _Pyx_Prefix(self):
        return "%s_%s_%s_"%(self._model, self._pyx_version, self._det)

    # ALL POSSIBLE MODELS WE USED IN OUR ANALYSES. 
    def allModels(self):
        return ['grbm', 'grbm+bbody', 'grbm+lpow', 'grbm+bbody+lpow',
                'sbpl', 'sbpl+bbody', 'sbpl+lpow', 'sbpl+bbody+lpow',
                'cutoffpl', 'cutoffpl+bbody', 'cutoffpl+lpow', 'cutoffpl+bbody+lpow']
    
    # ACCESSES THE GRB SAMPLE'S DATA, SUCH AS REDSHIFT, START AND ENDING TIMES. 
    def _BurstSample(self):
        '''
        _BurstSample(self)
        
        ACCESSES THE GRB SAMPLE'S DATA, SUCH AS REDSHIFT, START AND ENDING TIMES.
        Stored in the following directory/filename:
        /Users/KimiZ/Python/My_Modules/GRBAnalysis/data/burst_data.txt

        trigger, redshift, burstname, t90_start, t90_stop, BATdet, LATdet, angT0, angT30, angT60
        '''
        datafile        = os.path.join(self._base_dir, 'data/burst_data.txt')
        data			= pandas.read_table(datafile, delimiter=" ", header=0)
        #
        trigger			= str(data.trigger[data['trigger']== self._burst])
        redshift		= float(data.z[data['trigger']== self._burst])
        burstname		= str(data.name[data['trigger']== self._burst])
        t90_start		= float(data.t90_start[data['trigger']== self._burst])
        t90_stop		= float(data.t90_stop[data['trigger']== self._burst])
        BATdet			= data.BATdet[data['trigger']== self._burst]
        LATdet			= data.LATdet[data['trigger']== self._burst]
        angT0			= data.angT0[data['trigger']== self._burst]
        angT30			= data.angT30[data['trigger']== self._burst]
        angT60			= data.angT60[data['trigger']== self._burst]
        return trigger, redshift, t90_start, t90_stop, data

    # PROPERTIES TO EASILY ACCESS THE BURST SAMPLE DATA ABOVE. 
    @property
    def _burstdata(self):
        return self._BurstSample()[-1]
    
    @property
    def _trigger(self):
        return self._BurstSample()[0]

    @property
    def _redshift(self):
        return self._BurstSample()[1]

    @property
    def _t90start(self):
        return self._BurstSample()[2]

    @property
    def _t90stop(self):
        return self._BurstSample()[3]


    # GET PARAMETER NAMES BASED ON MODEL.
    def ParNames(self, model=None):
        '''
        ParNames(self, model=None)
        
        Given a model name, it will return the parameter names of the model.
        See file 'ParameterNames.json in the $PYX/Analysis/ directory.
        '
        '''
        if model:
            modname = model
        else:
            modname = self._model
        File  = os.path.join(self._basedata_dir, 'ParameterNames.json')
        f     = json.load(open(File, 'r'), encoding='utf-8')
        names = f[modname]
        names = [str(i) for i in names]
        return names


    def ParNames_ext(self, model=None):
        '''
        ParNames_ext(self, model=None)
        
        
        Takes the parameter names and extends them with the number of their order.
        ex:     alpha --> alpha__1
                beta --> beta__2

        These are used way more often than ParNames alone.
        '''
        if model:
            modname  = model
            parNames = self.ParNames(modname)
        else:
            parNames = self.ParNames()
        newNames = []
        for i,j in enumerate(parNames):
            newNames.append('%s__%i'%(j, i+1))
        return newNames

    def nPars(self, model=None):
        '''
        nPars(self, model=None)
        
        Returns the number of parameters for a model. 
         
        '''
        if model:
            return len(self.ParNames(model=model))
        else:
            return len(self.ParNames())


In [74]:
class Data(object):
    def __init__(self, *args, **kwargs):
        if args:
            raise Exception("Use Dictionary for now.")
        elif kwargs:
            self._burst           = kwargs['burst']
            self._det             = kwargs['det']
            if 'model' in kwargs.keys():
                self._model           = kwargs['model']
            if 'pyx_version' in kwargs.keys():
                self._pyx_version     = kwargs['pyx_version']
            if 'bxa_version' in kwargs.keys():
                self._bxa_version     = kwargs['bxa_version']
            self._bxa_Aversion    = '-01-'
            
        # DEFINE A FEW DIRECTORIES 
        self._det_dir             = ('GBMwLAT' if 'L' in self._det else 'GBM')
        self._base_dir            = "/Users/KimiZ/Python/My_Modules/GRBAnalysis/"
        self._basedata_dir        = os.path.join(self._base_dir, 'data/')
        self._burst_dir           = "/Users/KimiZ/GRBs2/analysis/LAT/%s/"%(self._burst)
    
    # ACCESSES THE GRB SAMPLE'S DATA, SUCH AS REDSHIFT, START AND ENDING TIMES. 
    def _BurstSample(self):
        datafile = os.path.join(self._base_dir, 'data/burst_data.txt')
        data     = pandas.read_table(datafile, delimiter=" ", header=0)
        colnames = data.colnames.tolist()
        #
        trigger			= str(data.trigger[data['trigger']== self._burst])
        redshift		= float(data.z[data['trigger']== self._burst])
        burstname		= str(data.name[data['trigger']== self._burst])
        t90_start		= float(data.t90_start[data['trigger']== self._burst])
        t90_stop		= float(data.t90_stop[data['trigger']== self._burst])
        BATdet			= data.BATdet[data['trigger']== self._burst]
        LATdet			= data.LATdet[data['trigger']== self._burst]
        angT0			= data.angT0[data['trigger']== self._burst]
        angT30			= data.angT30[data['trigger']== self._burst]
        angT60			= data.angT60[data['trigger']== self._burst]
        return trigger, redshift, t90_start, t90_stop, data

    
    # PROPERTIES TO EASILY ACCESS THE BURST SAMPLE DATA ABOVE. 
    @property
    def _burstdata(self):
        return self._BurstSample()[-1]
    
    @property
    def _burstdata(self):
        return self._BurstSample()[-1]
    
    @property
    def _trigger(self):
        return self._BurstSample()[0]

    @property
    def _redshift(self):
        return self._BurstSample()[1]

    @property
    def _t90start(self):
        return self._BurstSample()[2]

    @property
    def _t90stop(self):
        return self._BurstSample()[3]


params_dict = {
            'burst':        'bn080916009',
            'det':          'L',
            'model':        'grbm'}
                    
test        = Data(**params_dict)

In [ ]:
class Data(object):
    def __init__(self, *args, **kwargs):
        if args:
            raise Exception("Use Dictionary for now.")
        elif kwargs:
            self._burst           = kwargs['burst']
            self._det             = kwargs['det']
            if 'model' in kwargs.keys():
                self._model           = kwargs['model']
            if 'pyx_version' in kwargs.keys():
                self._pyx_version     = kwargs['pyx_version']
            if 'bxa_version' in kwargs.keys():
                self._bxa_version     = kwargs['bxa_version']
            self._bxa_Aversion    = '-01-'
            
        # DEFINE A FEW DIRECTORIES 
        self._det_dir             = ('GBMwLAT' if 'L' in self._det else 'GBM')
        self._base_dir            = "/Users/KimiZ/Python/My_Modules/GRBAnalysis/"
        self._basedata_dir        = os.path.join(self._base_dir, 'data/')
        self._burst_dir           = "/Users/KimiZ/GRBs2/analysis/LAT/%s/"%(self._burst)
        
    

    
    def get_data(self, column=None, burst=None):
        # default is to return full data table. 
        datafile = os.path.join(self._base_dir, 'data/burst_data.txt')
        data     = pandas.read_table(datafile, delimiter=" ", header=0)
        colnames = data.colnames.tolist()
        
        if column and burst:
            # returns a pandas series. A single result, so will need to use
            # str(v), float(v), etc. for the output. v is the output. 
            return data.loc[data.trigger == burst, column]#.values
        elif column and not burst:
            # returns a pandas series (a single column of the orig dataframe).
            return data.loc[:, column]#.values
        elif burst and not column:
            # returns pandas dataframe with specific GRB data. One row of data. 
            return data.loc[data.trigger == burst, :]#.values
        else:
            # returns pandas dataframe with ALL GRB data. 
            return data 

    
    # ACCESSES THE GRB SAMPLE'S DATA, SUCH AS REDSHIFT, START AND ENDING TIMES. 
    def _BurstSample(self, column=None, burst=None): 
        # default is to return full data table. 
        datafile = os.path.join(self._base_dir, 'data/burst_data.txt')
        data     = pandas.read_table(datafile, delimiter=" ", header=0)
        colnames = data.colnames.tolist()
        
        if column and burst:
            return data
        #
        trigger			= str(data.trigger[data['trigger']== self._burst])
        redshift		= float(data.z[data['trigger']== self._burst])
        burstname		= str(data.name[data['trigger']== self._burst])
        t90_start		= float(data.t90_start[data['trigger']== self._burst])
        t90_stop		= float(data.t90_stop[data['trigger']== self._burst])
        BATdet			= data.BATdet[data['trigger']== self._burst]
        LATdet			= data.LATdet[data['trigger']== self._burst]
        angT0			= data.angT0[data['trigger']== self._burst]
        angT30			= data.angT30[data['trigger']== self._burst]
        angT60			= data.angT60[data['trigger']== self._burst]
        return trigger, redshift, t90_start, t90_stop, data

    
    # PROPERTIES TO EASILY ACCESS THE BURST SAMPLE DATA ABOVE. 
    @property
    def _burstdata(self):
        return self._BurstSample()[-1]
    
    @property
    def _burstdata(self):
        return self._BurstSample()[-1]
    
    @property
    def _trigger(self):
        return self._BurstSample()[0]

    @property
    def _redshift(self):
        return self._BurstSample()[1]

    @property
    def _t90start(self):
        return self._BurstSample()[2]

    @property
    def _t90stop(self):
        return self._BurstSample()[3]


params_dict = {
            'burst':        'bn080916009',
            'det':          'L',
            'model':        'grbm'}
                    
test        = Data(**params_dict)

In [241]:
class Data(object):
    def __init__(self, *args, **kwargs):
        if args:
            raise Exception("Use Dictionary for now.")
        elif kwargs:
            self._burst           = kwargs['burst']
            self._det             = kwargs['det']
            if 'model' in kwargs.keys():
                self._model           = kwargs['model']
            if 'pyx_version' in kwargs.keys():
                self._pyx_version     = kwargs['pyx_version']
            if 'bxa_version' in kwargs.keys():
                self._bxa_version     = kwargs['bxa_version']
            self._bxa_Aversion    = '-01-'
            
        # DEFINE A FEW DIRECTORIES 
        self._det_dir             = ('GBMwLAT' if 'L' in self._det else 'GBM')
        self._base_dir            = "/Users/KimiZ/Python/My_Modules/GRBAnalysis/"
        self._basedata_dir        = os.path.join(self._base_dir, 'data/')
        self._burst_dir           = "/Users/KimiZ/GRBs2/analysis/LAT/%s/"%(self._burst)
        # ALL BASIC GRB DATA FOR OUR GRB SAMPLE. 
        # redshift, t90start, t90stop, etc. 
        self.burst_data       = pandas.read_table(os.path.join(self._base_dir, 'data/burst_data.txt'), 
                                       delimiter=" ", header=0)
    
    @property # using this means we don't need () after the call. 
    def get_BurstData_colnames(self):
        return self.burst_data.columns.tolist()

    
    def get_BurstData(self, column=None, burst=None):
        '''
        get_BurstData(column=None, burst=None)
        
        '''
        # default is to return full data table. 
        data = self.burst_data 
        if column and burst:
            # returns a pandas series. A single result, so will need to use
            # str(v), float(v), etc. for the output. v is the output. 
            return data.loc[data.trigger == burst, column]#.values
        elif column and not burst:
            # returns a pandas series (a single column of the orig dataframe).
            return data.loc[:, column]#.values
        elif burst and not column:
            # returns pandas dataframe with specific GRB data. One row of data. 
            return data.loc[data.trigger == burst, :]#.values
        else:
            # returns pandas dataframe with ALL GRB data. 
            return data 
        
    def output(self, format="clipboard"):
        '''
        output(format="clipboard")
        
        # format options:
        ['pickle', 'csv', 'json', 'clipboard', 'dict', 'sql']
        
        '''
        output_function = getattr(pandas.DataFrame, "to_%s" %format)
        return output_function(self.burst_data)  

    def output2(self, form="clipboard"):
        '''
        output(format="clipboard")
        
        # format options:
        ['pickle', 'csv', 'json', 'clipboard', 'dict', 'sql']
        
        '''
        output_function = getattr(pandas.DataFrame, "to_%s" %form)
        return output_function(self.burst_data)  
        
#     def output(self, format="text"):                              
#         output_function = getattr(statsout, "output_%s" %format)
#         return output_function(data)  

In [253]:
?format

In [242]:
params_dict = {
            'burst':        'bn080916009',
            'det':          'L',
            'model':        'grbm'}
test = Data(**params_dict)
# full data frame
test.burst_data.head()

,number,name,trigger,z,t90_start,t90_stop,BATdet,LATdet,angT0,angT30,angT60
0,0,GRB080804,bn080804972,2.2045,0.256,24.960,yes,no,0.0000,0.0000,0.0000
1,1,GRB080810,bn080810549,3.3604,-20.096,87.361,yes,no,0.0000,0.0000,0.0000
2,2,GRB080905A,bn080905499,0.1218,-0.064,0.896,yes,no,27.8855,29.1496,30.4518
3,3,GRB080905B,bn080905705,2.3739,-5.120,100.864,yes,no,85.4289,85.8448,86.2924
4,4,GRB080916A,bn080916406,0.6887,0.512,46.849,yes,no,75.8313,77.1288,78.4168


In [252]:
test.output('json')

'{"number":{"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"11":11,"12":12,"13":13,"14":14,"15":15,"16":16,"17":17,"18":18,"19":19,"20":20,"21":21,"22":22,"23":23,"24":24,"25":25,"26":26,"27":27,"28":28,"29":29,"30":30,"31":31,"32":32,"33":33,"34":34,"35":35,"36":36,"37":37,"38":38,"39":39,"40":40,"41":41,"42":42,"43":43,"44":44,"45":45,"46":46,"47":47,"48":48,"49":49,"50":50,"51":51,"52":52,"53":53,"54":54,"55":55,"56":56,"57":57,"58":58,"59":59,"60":60,"61":61,"62":62,"63":63,"64":64,"65":65,"66":66,"67":67,"68":68,"69":69},"name":{"0":"GRB080804","1":"GRB080810","2":"GRB080905A","3":"GRB080905B","4":"GRB080916A","5":"GRB080916C","6":"GRB080928","7":"GRB081008","8":"GRB081121","9":"GRB081221","10":"GRB081222","11":"GRB090102","12":"GRB090113","13":"GRB090323","14":"GRB090328A","15":"GRB090423","16":"GRB090424","17":"GRB090510","18":"GRB090519","19":"GRB090618","20":"GRB090902B","21":"GRB090926A","22":"GRB090926B","23":"GRB090927","24":"GRB091003","25":"GRB091020"

In [251]:
ls -ltr 

total 5328
-rw-r--r--  1 KimiZ  staff   12510 Apr 11  2016 python_tutorial_1.ipynb
-rw-r--r--  1 KimiZ  staff    3895 Apr 21  2016 Guess my number.ipynb
-rw-r--r--  1 KimiZ  staff    6250 Apr 21  2016 from the love-python blog.ipynb
-rw-r--r--  1 KimiZ  staff   81195 Apr 24  2016 python_basics_tutorial_1_Jupyter.ipynb
-rw-r--r--  1 KimiZ  staff   15698 May  1  2016 python_basics_tutorial_2_Jupyter.ipynb
-rw-r--r--  1 KimiZ  staff      91 May  2  2016 out.txt
-rw-r--r--  1 KimiZ  staff    3414 May  7  2016 will help with pyxspec.ipynb
-rw-r--r--  1 KimiZ  staff    7654 May  9  2016 Tkinter.ipynb
-rw-r--r--  1 KimiZ  staff    5265 May  9  2016 python tutorial Data Structures.ipynb
-rw-r--r--  1 KimiZ  staff   97057 May 11  2016 python_tutorial_ControlFlow_Jupyter.ipynb
-rw-r--r--  1 KimiZ  staff    5134 May 11  2016 Object Oriented Programming.ipynb
-rw-r--r--  1 KimiZ  staff      84 Jun 15  2016 workfile
-rw-r--r--  1 KimiZ  staff      87 Jun 15  2016 workfile_2
-rw-r--r--  1 KimiZ  sta

In [249]:
!open ./

In [223]:
test.get_BurstData_colnames

['number',
 'name',
 'trigger',
 'z',
 't90_start',
 't90_stop',
 'BATdet',
 'LATdet',
 'angT0',
 'angT30',
 'angT60']

In [224]:
test.burst_data.head(10)

,number,name,trigger,z,t90_start,t90_stop,BATdet,LATdet,angT0,angT30,angT60
0,0,GRB080804,bn080804972,2.2045,0.256,24.960,yes,no,0.0000,0.0000,0.0000
1,1,GRB080810,bn080810549,3.3604,-20.096,87.361,yes,no,0.0000,0.0000,0.0000
2,2,GRB080905A,bn080905499,0.1218,-0.064,0.896,yes,no,27.8855,29.1496,30.4518
3,3,GRB080905B,bn080905705,2.3739,-5.120,100.864,yes,no,85.4289,85.8448,86.2924
4,4,GRB080916A,bn080916406,0.6887,0.512,46.849,yes,no,75.8313,77.1288,78.4168
5,5,GRB080916C,bn080916009,4.3500,1.280,64.257,no,yes,48.8396,48.6589,48.4850
6,6,GRB080928,bn080928628,1.6919,-1.792,12.544,yes,no,45.2153,44.3400,43.4711
7,7,GRB081008,bn081008832,1.9670,0.004,150.019,yes,no,64.2697,64.9711,65.6619
8,8,GRB081121,bn081121858,2.5120,1.536,43.521,yes,no,134.7390,133.9690,133.1950
9,9,GRB081221,bn081221681,2.2600,3.328,33.025,yes,no,77.2159,78.9389,80.7400


In [225]:
test.burst_data.shape

(70, 11)

In [226]:
test.burst_data.tail(10)

,number,name,trigger,z,t90_start,t90_stop,BATdet,LATdet,angT0,angT30,angT60
60,60,GRB130612A,bn130612141,2.0060,-2.560,4.864,yes,no,44.3786,44.6751,45.0255
61,61,GRB130702A,bn130702004,0.1450,0.768,59.649,no,yes,76.4101,75.2035,73.9996
62,62,GRB130925A,bn130925173,0.3470,11.008,226.563,yes,no,36.3258,35.8424,30.2735
63,63,GRB131004A,bn131004904,0.7170,-0.192,0.960,yes,no,93.0052,93.9654,94.9436
64,64,GRB131105A,bn131105087,1.6860,2.496,115.138,yes,no,35.9647,36.0518,36.1368
65,65,GRB131108A,bn131108862,2.4000,0.448,18.944,no,yes,24.4113,28.4615,28.6986
66,66,GRB131231A,bn131231198,0.6420,13.312,44.544,no,yes,38.0867,39.1376,39.7363
67,67,GRB140206A,bn140206304,2.7400,0.512,27.776,yes,no,123.3730,123.3730,123.3730
68,68,GRB140213A,bn140213807,1.2076,0.768,19.392,yes,no,48.4950,48.4047,47.1975
69,69,GRB140304A,bn140304557,5.2830,-5.376,26.368,yes,no,72.5804,73.7782,74.9692


To get specific data for other bursts, you can either use the test.burst_data dataframe and located the desired element using .loc, or you can use test.get_BurstData. 

test.get_BurstData takes a column name and/or burst trigger id. 

In [192]:
test.get_BurstData(column='z', burst='bn080804972')

0    2.2045
Name: z, dtype: float64

In [193]:
test.get_BurstData(column='z', burst='bn080804972')

0    2.2045
Name: z, dtype: float64

In [194]:
test.get_BurstData(column='z').head(10) 

0    2.2045
1    3.3604
2    0.1218
3    2.3739
4    0.6887
5    4.3500
6    1.6919
7    1.9670
8    2.5120
9    2.2600
Name: z, dtype: float64

In [195]:
test.get_BurstData(burst='bn080804972')

,number,name,trigger,z,t90_start,t90_stop,BATdet,LATdet,angT0,angT30,angT60
0,0,GRB080804,bn080804972,2.2045,0.256,24.96,yes,no,0.0,0.0,0.0


In [196]:
test.burst_data.head(10)

,number,name,trigger,z,t90_start,t90_stop,BATdet,LATdet,angT0,angT30,angT60
0,0,GRB080804,bn080804972,2.2045,0.256,24.960,yes,no,0.0000,0.0000,0.0000
1,1,GRB080810,bn080810549,3.3604,-20.096,87.361,yes,no,0.0000,0.0000,0.0000
2,2,GRB080905A,bn080905499,0.1218,-0.064,0.896,yes,no,27.8855,29.1496,30.4518
3,3,GRB080905B,bn080905705,2.3739,-5.120,100.864,yes,no,85.4289,85.8448,86.2924
4,4,GRB080916A,bn080916406,0.6887,0.512,46.849,yes,no,75.8313,77.1288,78.4168
5,5,GRB080916C,bn080916009,4.3500,1.280,64.257,no,yes,48.8396,48.6589,48.4850
6,6,GRB080928,bn080928628,1.6919,-1.792,12.544,yes,no,45.2153,44.3400,43.4711
7,7,GRB081008,bn081008832,1.9670,0.004,150.019,yes,no,64.2697,64.9711,65.6619
8,8,GRB081121,bn081121858,2.5120,1.536,43.521,yes,no,134.7390,133.9690,133.1950
9,9,GRB081221,bn081221681,2.2600,3.328,33.025,yes,no,77.2159,78.9389,80.7400


In [197]:
test.burst_data.tail(10)

,number,name,trigger,z,t90_start,t90_stop,BATdet,LATdet,angT0,angT30,angT60
60,60,GRB130612A,bn130612141,2.0060,-2.560,4.864,yes,no,44.3786,44.6751,45.0255
61,61,GRB130702A,bn130702004,0.1450,0.768,59.649,no,yes,76.4101,75.2035,73.9996
62,62,GRB130925A,bn130925173,0.3470,11.008,226.563,yes,no,36.3258,35.8424,30.2735
63,63,GRB131004A,bn131004904,0.7170,-0.192,0.960,yes,no,93.0052,93.9654,94.9436
64,64,GRB131105A,bn131105087,1.6860,2.496,115.138,yes,no,35.9647,36.0518,36.1368
65,65,GRB131108A,bn131108862,2.4000,0.448,18.944,no,yes,24.4113,28.4615,28.6986
66,66,GRB131231A,bn131231198,0.6420,13.312,44.544,no,yes,38.0867,39.1376,39.7363
67,67,GRB140206A,bn140206304,2.7400,0.512,27.776,yes,no,123.3730,123.3730,123.3730
68,68,GRB140213A,bn140213807,1.2076,0.768,19.392,yes,no,48.4950,48.4047,47.1975
69,69,GRB140304A,bn140304557,5.2830,-5.376,26.368,yes,no,72.5804,73.7782,74.9692


In [38]:
params_dict = {
            'burst':        'bn080916009',
            'det':          'L',
            'model':        'grbm',
            'pyx_version':  '-01-',
            #'bxa_version':  '-01-',
            #'bxa_Aversion': '-01-'
            }
                    
test        = Data(**params_dict)

In [5]:
class Models:
    # class variables shared by all instances are defined here. 
    name = modname
#     parvalues = values  # class variables shared by all instances
#     parnames = parnames
    def band(self, alpha, beta, epeak, norm):
        self.alpha = -1.0
        self.beta = -2.0
        self.epeak = 300.0
        self.norm = 0.01
        def pars(self, x):
            vals = x*4
            return vals
    
#     def parameters(self, alpha, beta, epeak, norm):
#         self.alpha = 0.0
#         self.beta = 0.0
#         self.epeak = 0.0
#         self.norm = 0.0
        
#     def __init__(self, name):
#         self.name = 'zoldak'  # instance variable unique to each instance

NameError: name 'modname' is not defined

In [ ]:
#values = [-1.0, -2.0, 300.0, 0.01]
#pars = ['alpha','beta','epeak','norm']

In [ ]:
modname = 'band'

In [ ]:
Models.name

In [ ]:
Models.parnames

In [ ]:
Models.parvalues

In [ ]:
kz = Models()

In [ ]:
kz.band('1')

In [ ]:
class MyClass(object):
    def grades(self, x):
        
        return x

In [ ]:
D.f  # just an unbound method

In [ ]:
D.__dict__['f']  # stores internally as a function

In [ ]:
d= D()

In [ ]:
print type(d) # <class '__main__.D'>
print type(D) # <type 'type'>
print type(d.f) # <type 'instancemethod'>
print type(D.f) # <type 'instancemethod'>

In [ ]:
print d.f  # <bound method D.f of <__main__.D object at 0x104b2e450>>
print D.f  # <unbound method D.f>

D -- object, class or module name.
f -- attribute of the object, class or module.

attributes can be read only, or read and write.
D.number = 42 will make a new attribute (called number) and write it to the class (called D).

In [ ]:
D.number = 42

In [ ]:
D.number

#### This will also carry over to the bound method. But won't work the opposite way around.

In [ ]:
d.number

In [ ]:
d.letter = 'A'

In [ ]:
# D.letter  # not recognized.

d is bound to D and any changes made to D will be carried through to d.  But any changes made to d will not carry through to D.  d is bound to D.  

In [ ]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)

    __update = update   # private copy of original update() method

class MappingSubclass(Mapping):

    def update(self, keys, values):
        # provides new signature for update()
        # but does not break __init__()
        for item in zip(keys, values):
            self.items_list.append(item)

In [ ]:
list = [1,2,3,4,5]
range(len(list))

In [ ]:
Mapping.update([1,2,3])

In [ ]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def next(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [ ]:
rev = Reverse('spam')

In [ ]:
iter(rev)

In [ ]:
for char in rev:
    print char

In [ ]:
lname = Reverse('Zoldak')

for letter in lname:
    print letter

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10]

In [ ]:
numbers = Reverse(a)

for n in numbers:
    print n

In [ ]:
for n in iter(a):
    print n

## Nested functions and nested scopes

# Kim's Play

In [ ]:
class Model(object):
    # statements
    #name = 'grbm'
    #pars = ['alpha','beta','tem','norm']
    
    
    def grbm(self, name='name', parNames = [], default=[]):
        name = 'grbm'
        parNames = ['alpha', 'beta', 'tem', 'norm']
        default = [-1.0, -2.5, 300, 1]
        
        
        #self.name = 'grbm'
        #self.parNames = ['alpha', 'beta', 'tem', 'norm']
        #self.default = [-1.0, -2.5, 300, 1]
        #params = {"alpha":-1.235, "beta":-2.534, "tem": 555.2, "norm": 0.01}

        
    
    def alpha(self, value):
        self.value = -1.25
        return self.value
    
    
    
    
    # functions
    def __get__(self, value):
        self.value = -1.2
        return value
    
m1 = Model()

In [ ]:
class Model(object):
    # statements
    name = 'grbm'
    pars = ['alpha','beta','tem','norm']
    
    def alpha(self, value):
        self.value = -1.25
        return self.value
    
    
    
    
    # functions
    def __get__(self, value):
        self.value = -1.2
        return value
    
#fget = Model.__get__()

In [ ]:
m1.alpha('value')

In [ ]:
Model.pars

In [ ]:
Model.__get__()

In [ ]:
class MyClass(object):
    class_var = 1
    one = 1
    two = 2
    three = 3
    four = 4
    
    def alpha(self, alpha_var):
        self.alpha_var = alpha_var

    def __init__(self, i_var):
        self.i_var = i_var


In [ ]:
MyClass.__get__(object)

In [ ]:
MyClass.__init__

In [ ]:
MyClass.class_var

In [ ]:
MyClass.four

In [ ]:
class Model(object):
#     alpha = -1.2
#     beta = -2.5
#     tem = 322.5
#     norm = 1.0E-2

    def alpha(self, alpha_v):
        def name(self):
            return 'alpha'
        return
        
        
    
    def __get__(self, alpha, beta, tem, norm):
        self.alpha = alpha
        self.beta = beta
        self.tem = tem
        self.norm = norm

Model.__dict__['alpha']
    
    #grbm.alpha.values = [-1.02976, -0.0102976, -10.0, -3.0, 2.0, 5.0]
    #grbm.beta.values = [-2.20137, -0.022013699999999997, -10.0, -5.0, 2.0, 10.0]
    #grbm = {1:'alpha', 2:'beta', 3:'tem', 4:'norm'}
    

In [ ]:
Model.alpha('name')

In [ ]:
class Model(object):
    def showList(self):
        '''
         Additive Models: 
      agauss        apec       bapec       bbody    bbodyrad      bexrav
      bexriv     bkn2pow    bknpower         bmc      bremss      bvapec
     bvvapec      c6mekl     c6pmekl     c6pvmkl     c6vmekl      cemekl
      cevmkl       cflow      compLS      compPS      compST      compTT
      compbb     compmag      comptb      compth    cplinear    cutoffpl
        disk      diskbb      diskir    diskline       diskm       disko
     diskpbb      diskpn    eplogpar      eqpair     eqtherm       equil
      expdec    ezdiskbb       gadem    gaussian        gnei        grad
        grbm      kerrbb       kerrd    kerrdisk        laor       laor2
      logpar     lorentz        meka       mekal     mkcflow         nei
      nlapec     npshock         nsa     nsagrav     nsatmos       nsmax
      nsmaxg         nsx       nteea     nthComp     optxagn    optxagnf
    pegpwrlw      pexmon      pexrav      pexriv      plcabs        posm
    powerlaw      pshock     raymond       redge      refsch        rnei
       sedov        sirf       smaug       srcut       sresc        step
       vapec     vbremss      vequil      vgadem       vgnei     vmcflow
       vmeka      vmekal        vnei    vnpshock     vpshock    vraymond
       vrnei      vsedov      vvapec      vvgnei       vvnei   vvnpshock
    vvpshock      vvrnei     vvsedov     zagauss      zbbody     zbremss
      zgauss    zpowerlw

 Multiplicative Models: 
     SSS_ice       TBabs     TBgrain    TBvarabs      absori     acisabs
        cabs    constant     cyclabs        dust        edge      expabs
      expfac        gabs      heilin    highecut       hrefl       lyman
       notch      pcfabs       phabs       plabs        pwab      recorn
      redden      smedge    spexpcut      spline      swind1       uvred
      varabs      vphabs        wabs      wndabs        xion      zTBabs
       zbabs       zdust       zedge    zhighect        zigm     zpcfabs
      zphabs     zredden     zsmdust     zvarabs     zvfeabs     zvphabs
       zwabs     zwndabs      zxipcf

 Convolution Models: 
       cflux      cpflux     gsmooth    ireflect      kdblur     kdblur2
    kerrconv     lsmooth     partcov      rdblur     reflect     rgsxsrc
       simpl     zashift     zmshift

 Mixing Models: 
       ascac      projct      suzpsf      xmmpsf

 Pile-up Models: 
      pileup

 Additional models are available at :
	 legacy.gsfc.nasa.gov/docs/xanadu/xspec/newmodels.html
        '''
    def __get__(self, value):
        self.value = -1.25
        return self.value
    def __set__(self, value):
        self.value = value
        return
    def __delete__(self, value):
        del self.value
        return
    

#Model.__dict__['showList']
#Model.__setattr__['showList']
Model.__init__['showList']

In [ ]:
Model.__init__['showList']

Define any of these methods and an object is considered a descriptor and can override default behavior upon being looked up as an attribute.

In [ ]:
Model.__get__(self, alpha, type=None) 

In [ ]:
class Model(object):

    def __get__(self, alpha):
        alpha = -1.25
        return alpha

    def alpha(self, alpha_v):
        def name(self):
            return 'alpha'
        return
        
        
    
    def __get__(self, alpha, beta, tem, norm):
        self.alpha = alpha
        self.beta = beta
        self.tem = tem
        self.norm = norm

Model.__dict__['alpha']